# A1: Logic and lambda calculus

Simon Dobnik and Robin Cooper

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on how to work in groups on Canvas.

Write all your answers and the code in the appropriate boxes below.

*Important*: We recommend that you create a virtual environment (either with virtualenv or conda), install Jupyter Lab in it and all dependencies used in the assignment.
To run Jupyter Lab later within your environment, run the following command:

```python -m ipykernel install --user --name=my-virtualenv-name```,

where you replace `my-virtualenv-name` with the name of your created environment.
Once in Jupyter, choose the kernel with the name of your environment. You can do it by either (i) using the drop-down menu in the top right corner or (ii) going to the top menu -> Kernel -> Change Kernel.

## Translating English to logic and evaluating logic in a model

In [1]:
# This task needs NLTK and Jupyter Lab (IPython package).
import nltk 
from utils import display_latex, display_translation, display_tree, display, Markdown
read_expr = nltk.sem.Expression.fromstring

### 1. Propositional logic
Translate the following sentences into **propositional logic** and verify that they parse with Expression.fromstring() (`read_expr` variable in the cell above). Provide a key which shows how the propositional variables in your translation correspond to expressions of English. Briefly discuss any difficulties you encounter. (By difficulties we mean cases where the semantics of English expressions cannot be expressed to the same degree by the semantics of your logic representations, i.e. they do not mean the same). **[5 + 1 marks]**

In [ ]:
propositions = {
    "Alex plays the piano":
    read_expr('p'),

    "Alex is smart":
    read_expr('q'),

    "If Alex plays the piano, she is smart.":
    read_expr('p -> q'),
    
    "Alex is musical":
    read_expr('m'),

    "Alex is both smart and musical.":
    read_expr('q & m'),
    
    "Lydia is not happy":
    read_expr('-h'),
    
    "If Alex is not smart, Lydia is not happy.":
    read_expr('-q -> -h'),
    
    "George plays the piano":
    read_expr("g"),

    "George is musical":
    read_expr("l"),

    "If Alex or George plays the piano, they are musical.":
    read_expr('(p -> m) | (g -> l)'),
    
    "George plays the piano.":
    read_expr('g'),
}

for text, semrep in propositions.items():
    display_translation(text, semrep)

*Difficulties encountered:*

The statement "If Alex or George plays the piano, they are musical." It certainly posed some challenges.
For example, the formula "(p -> m) | (g -> l)"  could account for both of them playing the piano, as that would translate into ‘If Alex plays, then Alex is musical’  OR ‘If George plays, then George is musical.’ But it doesn’t fully capture the meaning. Let’s consider the following: If Alex plays, but isn’t musical,the statement will still be true just because George didn’t play. It lacks distributive meaning.
The other alternative would be (PA | PG) -> (MA & MG) which translates to “If Alex plays OR George plays, then Alex is musical AND George is musical.
This formula would require the consequent to be true (MA & MG) : both Alex and George must be musical, even though George didn't play.

Translating only with propositional letters and not predicates is challenging and highlights how linguistic meaning does not matter in logic, the only thing that matters is truth values.


### 2. Valuation of Propositional logic

Imagine that we observe a world where 
- (i) Alex does not play the piano,
- (ii) Alex and Lydia are smart and musical,
- (iii) George is not musical,
- (iv) Lydia is happy,
- (v) George plays the piano. 

Translate this informal description of the world into a model by appropriately defining an evaluation function and evaluate the formulae from Question 1 in this model. Briefly comment the answers you get. **[5 + 1 marks]**.

In [ ]:
value = {
    'A': False,   # Alex does not play piano
    'B': True,    # Alex is smart
    'C': True,    # Alex is musical
    'D': True,    # Lydia is happy
    'E': True,    # George plays piano
    'F': False,   # George is not musical
}

expressions = [
    "A -> B",              # If Alex plays the piano, she is smart.
    "B & C",               # Alex is both smart and musical.
    "-B -> -D",            # If Alex is not smart, Lydia is not happy.
    "(A | E) -> (C & F)",  # If Alex or George plays the piano, they are musical.
    "E"                    # George plays the piano.
]

def evaluate_value(expression, value):
    if expression.startswith('-'):
        var = expression[1:].strip()
        return not value.get(var, False)
    
    if '&' in expression:
        left, right = expression.split('&')
        left = left.strip()
        right = right.strip()
        return value.get(left, False) and value.get(right, False)
    
    if '|' in expression:
        left, right = expression.split('|')
        left = left.strip()
        right = right.strip()
        return value.get(left, False) or value.get(right, False)
    
    #A → B = ¬A ∨ B
    if '->' in expression:
        left, right = expression.split('->')
        left = left.strip()
        right = right.strip()
        return not value.get(left, False) or value.get(right, False)
    
    return value.get(expression.strip(), False)

for expr in expressions:
    result = evaluate_value(expr, value)
    print(f"{expr} : {result}")

*Comments:*

"If Alex plays the piano, she is smart."   This statement is true according to the model since Alex does not play the piano(-p) and Alex is smart(q). Thus, as the antecedent is false and the consequent is true, this means that the expression can be considered true as well.

"Alex is both smart and musical." this statement is true because according to the model, Alex is smart and musical (q & m). 

"If Alex is not smart, Lydia is not happy." this statement is vacuously true. The model states that  Alex is smart(q), which would mean that the antecedent “Alex is not smart” is false(-q). The model also states that Lydia is happy (h), thus the consequent 'Lydia is not happy' (-h) is False.  The implication  '-q -> -h' is valid.

"If Alex or George plays the piano, they are musical." This statement is true because George plays the piano(g) and this implication has the condition that either Alex or George have to  satisfy the requirement of playing the piano in order to be valid.

"George plays the piano" This statement is true, as George does indeed play the piano. Proposition (g) is assigned the true value.

### 3. Predicate logic *without quantifiers*

Translate the following sentences into predicate-argument formulae of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [ ]:
sentences1 = {
    "Lydia likes George but Lydia doesn't like Alex": 
    read_expr(r'like(lydia, george) & -like(lydia, alex)'),
    
    "Lydia likes herself and so does George":
    read_expr(r'like(lydia, lydia) & like(george, george)'),
    
    "Charlie is an English pianist who plays a sonata":
    read_expr(r'english(charlie) & pianist(charlie) & play(charlie, sonata)'),

    
    "Lydia and George admire each other":
    read_expr(r'admire(lydia, george) & admire(george, lydia)'),
}

for text, semrep in sentences1.items():
    display_translation(text, semrep)

*Difficulties encountered:*

The second sentence “Lydia likes herself and so does George” is a bit ambiguous due to the scope “so does”, which can either refer to liking oneself or to liking Lydia. It can be interpreted either as 'likes(lydia, lydia) & likes(george, lydia)' or likes(lydia, lydia) & likes(george, george).
  
 In the third sentence, treating "sonata" as an entity would be inappropriate, since "sonata" is a property, rather than a single identifiable entity. Therefore, we should use an existential quantifier to express the statement Charlie plays a sonata. In this case,the correct logical formula would be : ('exists x.sonata(x) & plays(charlie, x)').

### 4. First order logic with quantifiers

Translate the following sentences into quantified formulas of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [ ]:
sentences2 = {
    "Charlie knows a woman who likes George":
    read_expr('exists x.(woman(x) & like(x, george) & know(charlie, x))'),
    
    "George admires everybody and Lydia admires nobody":
    read_expr('all x.(person(x) -> (admire(george, x) & -admire(lydia, x)))'),

    "Nobody admires everybody":
    read_expr('-exists x.(person(x) & all y.(person(y) -> admire(x, y)))'),
    
    "Exactly one musician plays everything Alex wrote":
    read_expr('exists x.(musician(x) & all y.(wrote(alex, y) -> play(x, y)) & all z.(musician(z) & all y.(wrote(alex, y) -> play(z, y)) -> x=z))'),
}

for text, semrep in sentences2.items():
    display_translation(text, semrep)

*Difficulties encountered:*

The third sentence, "Nobody admires everybody", could  be alternatively expressed  as the statement "For all individuals x, it is not the case that there exists an individual y such that y admires x". However, expressing the sentence in this manner would be considered an unconventional or a misrepresentation of the intended meaning. A more suited interpretation would  be better conveyed through “ there isn’t a single person in this world that admires everyone”


### 5. Valuation of first order logic

We observe a world with entities Lydia, George, Alex, Charlie and Bertie, sonata, etude, prelude, waltz, scherzo.

1. Lydia likes Lydia, George, Alex and Charlie. George likes Lydia, Bertie and George. Alex likes Alex. Charlie likes Lydia, George, Alex, Charlie and Bertie. Bertie likes Alex.
2. Lydia, George, Alex, Charlie and Bertie are English.
3. Charlie and Bertie are pianists.
4. Charlie plays a sonata, an etude and a waltz. Bertie plays a waltz and a scherzo. Lydia plays an etude, a prelude and a waltz.
5. Lydia admires Lydia, Charlie and Bertie. George admires Lydia, George, Alex, Charlie and Bertie. Alex admires Lydia, Alex and Bertie. Charlie admires George and Bertie. Bertie admires Lydia, George, Alex, Charlie and Bertie.
6. Lydia knows Lydia, George, Alex, Charlie and Bertie. George knows Lydia, George and Bertie. Alex knows Lydia, Alex and Bertie. Charlie knows George, Charlie and Bertie. Bertie knows Lydia, George, Alex, Charlie and Bertie.
7. Lydia, Alex and Charlie are women.
8. George and Bertie are men.
9. Alex wrote a sonata, an etude an a waltz.
10. Lydia, Alex, Charlie and Bertie are musicians.

Translate this informal description of the world into a model and evaluate the formulae from Questions 3 and 4 in this model. Briefly comment on the answers you get **[3 + 2 marks]**.

In [ ]:
entities = set(['l','g','a','c','b','s','e','p','w','z'])

assign = """
    lydia => l
    george => g
    alex => a
    charlie => c
    bertie => b
    sonata => s
    etude => e
    prelude => p
    waltz => w
    scherzo => z
    english => {l, g, a, c, b}
    person => {l, g, a, c, b}
    musician => {l, a, c, b}
    pianist => {c, b}
    woman => {l, a, c}
    man => {g, b}
    like => {(l, l), (l, g), (l, a), (l, c), (g, l), (g, b), (g, g), (a, a), (c, l), (c, g), (c, a), (c, c), (c, b), (b, a)}
    admire => {(l, l), (l, c), (l, b), (g, l), (g, g), (g, a), (g, c), (g, b), (a, l), (a, a), (a, b), (c, g), (c, b), (b, l), (b, g),(b, a) (b, c), (b, b)}
    know => {(l, l), (l, g), (l, a), (l, c), (l, b), (g, l), (g, g), (g, b), (a, l), (a, a), (a, b), (c, g), (c, c), (c, b), (b, l), (b, g), (b, a), (b, c), (b, b)}
    play => {(c, s), (c, e), (c, w), (b, w), (b, z), (l, e), (l, p), (l, w)}
    wrote => {(a, s), (a, e), (a, w)}
"""

val2 = nltk.Valuation.fromstring(assign)

g2 = nltk.Assignment(entities)
m2 = nltk.Model(entities, val2)

# sentences from question 3
for text, semrep in sentences1.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

# sentences from question 4
for text, semrep in sentences2.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

*Comments on the answers:*

We encountered difficulties with some expressions because the quantifiers turned up as undefined,which might be the result of missing parenthesis. Could be related to the quantifier scope being undefined.

Translation of "Nobody admires everybody" as "all x. exists y. -admire(x,y)" doesn't work and we think it's because without the person() predicate objects like scherzo and etude are evaluated as x and y so it's never the case someone like everyone.

## Lambda calculus

In [7]:
from nltk.grammar import FeatureGrammar

### 6. Function application and $\beta$-reduction
In the following examples some code has been deleted and replaced with `<????>`. What has been deleted? Verify that your answer is correct. **[4 marks]**

In [ ]:
e1 = read_expr(r'\x.like(x,rob)')
e2 = read_expr(r'pip')
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result like(pip,rob).
display_latex(read_expr(r"like(pip,rob)"))

e1 = read_expr(r'\P.P(pip)')
e2 = read_expr(r'\x.play(x,scherzo)')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result play(pip,scherzo).
display_latex(read_expr(r"play(pip,scherzo)"))

e1 = read_expr(r'\P.exists x.(woman(x) & P(x))')
e2 = read_expr(r'\x.play(x,etude)') 
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result exists x.(woman(x) & play(x,etude)).
display_latex(read_expr(r"exists x.(woman(x) & play(x,etude))"))

e1 = read_expr(r'\Q.\x.Q(\y.like(x,y))')
e2 = read_expr(r'\P.all x. (musician(x) -> P(x))')
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result \x.all z2.(musician(z2) -> like(x,z2)).
display_latex(read_expr(r"\x.all z2.(musician(z2) -> like(x,z2))"))

Notes:

The path we took doing this exercise was to reverse engineer from the whole completed sentence and give unknown variables a lambda expression step by step. For example, we have started with the natural language sentence correspondence of montague grammars: "There exists a person x who is a woman and that person x plays etude". Then according to expressions given in e2, we have diminished the known entities and gave lambda expressions to the un-mentioned expression in e2 for a successful beta reduction. Such as, since e2 mentiones "There is an x person(entity) who plays etude." The lacking expression from the whole completed sentence is a proposition that says: "There exists an x. This e2 proposition is true under the circumstances that x is a woman." Which becomes our answer for e1.

The last exercise was the most challenging since there was multiple lambda expressions to assign. And correctly naming the variables took some trial and error.

### 7. Extending the grammar

Extend the grammar simple_sem.fcfg that comes with NLTK `(~/nltk_data/grammars/book_grammars/)` so that it will cover the following sentences:

- no man gives a bone to a dog **[4 marks]**
- no man gives a bone to the dog **[4 marks]**
- a boy and a girl chased every dog **[2 marks]**
- every dog chased a boy and a girl **[2 marks]**
- a brown cat chases a white dog **[4 marks]**

The last example includes adjectives. Several different kinds of adjectives are discussed in the literature [(cf. Kennedy, 2012)](http://semantics.uchicago.edu/kennedy/docs/routledge.pdf). In this example we have an intersective adjective. The denotiation we want for "brown cat" is a a set that we get by intersecting the set of individuals that are brown and the set of individuals that are cats.

C. Kennedy. Adjectives. In G. Russell, editor, The Routledge Companion to Philosophy of Language, chapter 3.3, pages 328–341. Routledge, 2012.

The original grammar is included in the code below as a string.

In [9]:
fcfg_string_orginal = r"""
% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'
 
Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'

N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.boy(x)>] -> 'boy'
N[NUM=sg,SEM=<\x.bone(x)>] -> 'bone'
N[NUM=sg,SEM=<\x.ankle(x)>] -> 'ankle'
N[NUM=sg,SEM=<\x.dog(x)>] -> 'dog'
N[NUM=pl,SEM=<\x.dog(x)>] -> 'dogs'

IV[NUM=sg,SEM=<\x.bark(x)>,TNS=pres] -> 'barks'
IV[NUM=pl,SEM=<\x.bark(x)>,TNS=pres] -> 'bark'
IV[NUM=sg,SEM=<\x.walk(x)>,TNS=pres] -> 'walks'
IV[NUM=pl,SEM=<\x.walk(x)>,TNS=pres] -> 'walk'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chases'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chase'
TV[NUM=sg,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'sees'
TV[NUM=pl,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'see'
TV[NUM=sg,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'give'

P[+to] -> 'to'
"""

Write your extension of this grammar here:

In [10]:
fcfg_string = fcfg_string_orginal + r"""
Det[NUM=sg,SEM=<\P Q.-(exists x.(P(x) & Q(x)))>] -> 'no'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x) & all y.(P(y) -> (y = x)))>] -> 'the'
CONJ[SEM=<\P B.\x.(P(x) & Q(x))>] -> 'and'
NP[NUM=pl,SEM=<\P.(?np1(P) & ?np2(P))>] -> NP[SEM=?np1] CONJ[SEM=?conj] NP[SEM=?np2]
N[NUM=sg,SEM=<\x.cat(x)>] -> 'cat'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'
ADJ[SEM=<\x.brown(x)>] -> 'brown'
ADJ[SEM=<\P x.(white(x) & P(x))>] -> 'white'
NP[NUM=?n,SEM=<?det(\x.(?adj(x) & ?noun(x)))> ] -> Det[NUM=?n, SEM=?det] ADJ[SEM=?adj] N[NUM=?n,SEM=?noun]
"""

# Load `fcfg_string` as a feature grammar:
syntax = FeatureGrammar.fromstring(fcfg_string)

Notes:

For this part, we started by replicating and making connections with what has been given in the original grammar. We think that "no" is the negation of "a" and "an" quantifiers and "the" is similar in structure with the addition of uniqueness. We achieved that by assigning a universal quantifier variable y that is true under the same circumstances when y equals to x. 

We have failed to define the phrase structure of ADJ containing phrase at first. It was the most challenging. We tried to define them as NP that contain Det, ADJ and N. (It turns out we had to define the SEM structure, det governing adj and noun with x argument)

(Also we have been writing 'n' instead of 'noun' the whole time.........)

Run the code below without errors:

In [ ]:
# comment out sentences if you couldn't find an answer for them
sentences = [
    'no man gives a bone to a dog',
    'no man gives a bone to the dog',
    'a boy and a girl chased every dog',
    'every dog chased a boy and a girl',
    'a brown cat chases a white dog',
]
for results in nltk.interpret_sents(sentences, syntax):
    for (synrep, semrep) in results:
        display(Markdown('----'))
        display_latex(semrep) # prints the SEM feature of a tree
        display_tree(synrep) # show the parse tree

If you are working with iPython which is also running behind Jupyter notebooks and you are changing grammars and want to rerun a new version without restarting you may find `nltk.data.clear_cache()` useful.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

The assignment is marked on a 7-level scale where 4 is sufficient to complete the assignment; 5 is good solid work; 6 is excellent work, covers most of the assignment; and 7: creative work. 

This assignment has a total of 47 marks. These translate to grades as follows: 1 = 17% 2 = 34%, 3 = 50%, 4 = 67%, 5 = 75%, 6 = 84%, 7 = 92% where %s are interpreted as lower bounds to achieve that grade.